# BERT 基本動作
BERT (Bidirectional Encoder Representations from Transofmers) は NLP の事前学習 (Pre-Trained) モデルです。文の単語それぞれの Embedding (埋め込み表現) を出力します。また、word2vec などの既存手法と異なり、文脈を考慮するのと、いわゆる一般的な深層学習モデルと同様にモデルの目的に応じて柔軟にネットワークが変更できるというメリットがあります。

本ノートブックでは、事前学習で用いられる `Mask` と `NSP` から BERT がどのような学習をしているのかを理解します。また、文脈を考慮した単語ベクトル (Word Embedding) を出力します。


## 必要なライブラリのインポート

HuggingFace の Transformers を使っていきます。

In [ ]:
import os
import torch
from torch import nn
import torch.nn.functional as F

from transformers import BertModel,BertJapaneseTokenizer, BertForMaskedLM, BertForNextSentencePrediction

## モデルの準備
学習済みモデルの tokenizer と model を取得します。

In [ ]:
# tokenizer と model の取得
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model.eval()

# Mask

一部の単語を MASK し MASK された箇所に入る単語を予測するタスク

In [ ]:
# オリジナルの文章
text = 'クラウド上で機械学習を行ってモデルを作成する。'
tokenized = tokenizer.tokenize(text)
print(tokenized)

In [ ]:
# MASK
masking_index = 11
tokenized[masking_index] = '[MASK]'
print(tokenized)

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokenized)
input_tensor = torch.tensor([ids])
with torch.no_grad():
    output = model(input_tensor)
    predictions = output[0][0, masking_index].topk(5)

In [ ]:
for k, i in enumerate(predictions.indices):
    index = i.item()
    word = tokenizer.convert_ids_to_tokens([index])[0]
    print(k, word)

# NSP

2 つの文章が連続した文章かどうかを当てるタスク

In [ ]:
model = BertForNextSentencePrediction.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model.eval()

In [ ]:
prompt = "東京都で4月15日、新たに729人が新型コロナウイルスに感染していることがわかりました。"
next_sentence = "マイクロソフトは、Azure Active Directory Basic および Premium サービスについて、99.99% の可用性を保証します。"
#next_sentence = "東京都は今後もPCR検査拡充を進めていくとともに、感染経路の洗い出しを行う方針です。"
encoding = tokenizer(prompt, next_sentence, return_tensors='pt')

In [ ]:
with torch.no_grad():
    outputs = model(**encoding, labels=torch.LongTensor([1]))
    logits = outputs.logits# %%print(outputs.logits)
    result = F.softmax(logits, dim=-1)

result = result.to('cpu').detach().numpy().copy()[0]
print("True: "+str(result[0]))
print("False: "+str(result[1]))

# Vector

文章から単語ベクトル Word Embedding を出力します。

In [ ]:
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model.eval()

In [ ]:
text = 'クラウド上で機械学習を行ってモデルを作成する。'
ids = tokenizer(text,padding=True,truncation=True,return_tensors="pt")
print(ids)

In [ ]:
with torch.no_grad():
    outputs = model(**ids).last_hidden_state # 最終隱れ層
    vec = outputs[:,0,:]
    vec = vec.view(-1, 768)

In [ ]:
print(vec.shape)
print(vec)